In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

In [2]:
df=pd.read_csv('data_new.csv')

In [3]:
data=df.drop(['FirstPaymentDate', 'FirstTimeHomebuyer', 'MaturityDate','MSA',\
             'Occupancy', 'LTV',  'Channel', 'PPM', 'ProductType', 'PropertyState',\
       'PropertyType', 'PostalCode', 'LoanSeqNum', 'LoanPurpose',\
       'OrigLoanTerm', 'NumBorrowers', 'SellerName', 'ServicerName', 'MonthsDelinquent', 'MonthsInRepayment', 'IsFirstTime'], axis=1)

In [4]:
data.head()

,CreditScore,MIP,Units,OCLTV,DTI,OrigUPB,OrigInterestRate,EverDelinquent,Years in Repayment,Repay_range_in_years,CreditRange,LTV_range
0,850,25,1,89,27,117000,6.750,0,4.3,4-10yrs,Excellent,High
1,850,0,1,73,17,109000,6.500,0,12.0,10-15yrs,Excellent,Medium
2,850,0,1,75,16,88000,6.875,0,5.6,4-10yrs,Excellent,Medium
3,850,0,1,76,14,160000,6.875,0,2.9,0-4yrs,Excellent,Medium
4,850,0,1,78,18,109000,7.125,0,4.5,4-10yrs,Excellent,Medium


In [5]:
data_with_dummies=pd.get_dummies(data,drop_first=True)
data_with_dummies.head()

x=data_with_dummies.drop(['EverDelinquent'], axis=1)
y=data_with_dummies['EverDelinquent']

In [6]:
from imblearn.over_sampling import SMOTE
ros = SMOTE()
X_res, y_res = ros.fit_resample(x, y)

In [8]:
# Split the dataset
x_train,x_test,y_train,y_test=train_test_split(X_res,y_res,test_size=0.1, random_state=1)

In [9]:
x_test.head()

,CreditScore,MIP,Units,OCLTV,DTI,OrigUPB,OrigInterestRate,Years in Repayment,Repay_range_in_years_10-15yrs,Repay_range_in_years_15-20yrs,Repay_range_in_years_4-10yrs,CreditRange_Fair,CreditRange_Good,CreditRange_Poor,LTV_range_Low,LTV_range_Medium
324725,671,30,1,95,35,164000,7.013172,5.084230,0,0,1,1,0,0,0,0
175817,763,0,1,75,40,150000,6.625000,4.400000,0,0,1,0,0,0,0,1
262843,684,0,1,77,33,176000,7.047205,3.075527,0,0,0,1,0,0,0,1
82597,699,25,1,90,33,144000,6.625000,12.400000,1,0,0,1,0,0,0,0
301803,742,0,1,73,33,155000,6.533326,3.759424,0,0,0,0,1,0,0,1


In [10]:
import numpy as np
x_train['CreditScore']=np.log(x_train['CreditScore']+1)
x_train['MIP']=np.log(x_train['MIP']+1)
x_train['OCLTV']=np.log(x_train['OCLTV']+1)
x_train['DTI']=np.log(x_train['DTI']+1)
x_train['OrigUPB']=np.log(x_train['OrigUPB']+1)
x_train['OrigInterestRate']=np.log(x_train['OrigInterestRate']+1)
x_train['Years in Repayment']=np.log(x_train['Years in Repayment']+1)

In [11]:
x_test['CreditScore']=np.log(x_test['CreditScore']+1)
x_test['MIP']=np.log(x_test['MIP']+1)
x_test['OCLTV']=np.log(x_test['OCLTV']+1)
x_test['DTI']=np.log(x_test['DTI']+1)
x_test['OrigUPB']=np.log(x_test['OrigUPB']+1)
x_test['OrigInterestRate']=np.log(x_test['OrigInterestRate']+1)
x_test['Years in Repayment']=np.log(x_test['Years in Repayment']+1)

In [12]:
# Importing the relevent module for performing LogisticRegression
from sklearn.linear_model import LogisticRegression
logisticReg=LogisticRegression()

logisticReg.fit(x_train, y_train)

LogisticRegression()

In [13]:
# The bias or intercept for the model
logisticReg.intercept_

array([0.96453183])

In [14]:
# Predicted values from the Logistic Regression
y_pred_train=logisticReg.predict(x_train)
y_pred_train

array([1, 0, 1, ..., 0, 1, 0], dtype=int64)

In [15]:
confusion_matrix(y_train,logisticReg.predict(x_train))

array([[115381,  39173],
       [ 47740, 106899]], dtype=int64)

In [16]:
print(classification_report(y_train,logisticReg.predict(x_train)))

              precision    recall  f1-score   support

           0       0.71      0.75      0.73    154554
           1       0.73      0.69      0.71    154639

    accuracy                           0.72    309193
   macro avg       0.72      0.72      0.72    309193
weighted avg       0.72      0.72      0.72    309193



In [23]:
import pickle
file=open("model.pkl", 'wb')
pickle.dump(logisticReg, file)

In [18]:
y_pred_test=logisticReg.predict(x_test)

In [21]:
print(classification_report(y_test,logisticReg.predict(x_test)))

              precision    recall  f1-score   support

           0       0.71      0.75      0.73     17220
           1       0.73      0.70      0.71     17135

    accuracy                           0.72     34355
   macro avg       0.72      0.72      0.72     34355
weighted avg       0.72      0.72      0.72     34355

